# Router

Because the SQLDatabaseChain were involked every single time, we seek to route it to a generic LLMChain when the query does not require querying the database. Here's our failed attempt.

In [1]:
import ast
import sys
from pathlib import Path
import warnings

from dotenv import load_dotenv


sys.path.append('..')
load_dotenv()

import duckdb
from duckdb_engine import DuckDBEngineWarning
warnings.filterwarnings("ignore", category=DuckDBEngineWarning)

In [2]:
from langchain.chains.router import MultiPromptChain
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chains import ConversationChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chains.llm import LLMChain

from langchain.chat_models import ChatOpenAI
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts import PromptTemplate

from app.usecases.sql import SQLDatabaseSequentialChain
from app.usecases.multi_sql import MultiRouteChain, MultiSQLChain
from app.database import DATABASE

gpt4_llm = ChatOpenAI(
    model_name='gpt-4-32k-0613',
    temperature=0,
    streaming=True
)
sql_dastabase_chain = SQLDatabaseSequentialChain.from_llm(
    llm=gpt4_llm,    
    database=DATABASE,
    verbose=True
)

gpt3_llm = ChatOpenAI(
    model_name='gpt-3.5-16k-0613',
    temperature=0,
    streaming=True
)
meta_chain = ConversationChain(
    llm=gpt3_llm,
    output_key="Answer:"
)

destination_chains = {
    # "meta": meta_chain,
    "sql_database": sql_dastabase_chain,
}

In [3]:

destinations = [
    # """meta: Good for answering questions without running SQL queries""",
    """sql_database: Only used for querying database for answers."""
]
destinations_str = "\n".join(destinations)

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)


In [4]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
sql_database: Only used for querying database for answers.

<< INPUT >>
{input}

<< OUTPUT >>



In [5]:
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

In [6]:
router_chain = LLMRouterChain.from_llm(gpt3_llm, router_prompt)

In [7]:
chain = MultiSQLChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=meta_chain,
    verbose=True,
)

ValidationError: 4 validation errors for MultiSQLChain
destination_chains -> sql_database -> llm_chain
  field required (type=value_error.missing)
destination_chains -> sql_database -> database
  field required (type=value_error.missing)
destination_chains -> sql_database -> decider_chain
  extra fields not permitted (type=value_error.extra)
destination_chains -> sql_database -> sql_chain
  extra fields not permitted (type=value_error.extra)

# Abort!!

Seems like Langchain does not like routing between regular Chains and SQLDatabaseChains, we decided to abort the effort of writing custom classes and use SQLAgent instead.